In [401]:
from aocd import get_data, submit
import numpy as np
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

DIRECTIONS_4 = [(x, y) for x in [1, 0, -1] for y in [1, 0, -1] if x + y and (x == 0 or y == 0)]
DIRECTIONS_8 = [(x, y) for x in [1, 0, -1] for y in [1, 0, -1] if not (x ==0 and y == 0)]

def raw_read_input(day):
    return get_data(day=day, year=2023, block=True)

def read_input(day, dtype=None):
    lines = raw_read_input(day=day).splitlines()
    if dtype is not None:
        lines = [dtype(x) if x else None for x in lines]
    return lines
    
def read_matrix(day, dtype=np.int32):
    lines = read_input(day)
    lines = [[dtype(x) for x in line] for line in lines]
    return np.array(lines, dtype=dtype)

# Day 1

In [220]:
lines = read_input(day=1)
digits_per_line = [[char for char in line if char.isdigit()] for line in lines]

def calc_sum(for_lines):
    return sum(int(line[0] + line[-1]) if line else 0 for line in for_lines)
print('part 1', calc_sum(digits_per_line))

def replace_digits(for_line):
    digit_names = ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']
    result = ''
    for index, char in enumerate(for_line):
        for digit, digit_name in enumerate(digit_names):
            if char.isdigit():
                result += char
            elif for_line[index: index + len(digit_name)] == digit_name:
                result += str(digit)
    return result
        
true_digits_per_line = [replace_digits(line) for line in lines]
print('part 2', calc_sum(true_digits_per_line))

part 1 54597
part 2 54504


# Day 2

In [54]:
from dataclasses import dataclass, field
from enum import Enum
from typing import Dict, List
from functools import reduce
from operator import mul

class Color(Enum):
    red = 'red'
    green = 'green'
    blue = 'blue'

@dataclass
class Game:
    game_number: int
    shows: List[Dict[Color, int]] = field(default_factory=list)

    def is_possible(self, combination: Dict[Color, int]) -> bool:
        for show in self.shows:
            for color in Color:
                if show[color] > combination[color]:
                    return False
        return True

    def power_number(self):
        possible = {color: 0 for color in Color}
        for show in self.shows:
            for color, amount in show.items():
                possible[color] = max(possible[color], amount)
        return reduce(mul, possible.values())


lines = read_input(day=2)
games = []

for line in lines:
    game_title, showings = line.split(':')
    game = Game(int(game_title.split(' ')[1]))

    for show in showings.split(';'):
        current_show = {color: 0 for color in Color}
        for color_group in show.split(', '):
            count, color = color_group.strip().split(' ')
            current_show[Color(color)] = int(count)
        game.shows.append(current_show)
    games.append(game)

part_1_check = {
    Color.red: 12,
    Color.green: 13,
    Color.blue: 14
}
print('part 1', sum(game.game_number for game in games if game.is_possible(part_1_check)))
print('part 2', sum(game.power_number() for game in games))

part 1 2331
part 2 71585


# Day 3

In [167]:
from collections import defaultdict
matrix = read_matrix(day=3, dtype=str)
expand = np.pad(matrix, (1, 1), constant_values=['.'])
is_symbol = np.vectorize(lambda x: not x.isdigit() and x != '.')(expand)
is_star = expand == '*'

gears = defaultdict(list)
total = 0

for i, line in enumerate(expand):
    number_start = None
    for j, char in enumerate(line):
        if char.isdigit():
            if number_start is None:
                number_start = j
        elif number_start is not None:
            if np.any(is_symbol[i - 1: i + 2, number_start - 1: j + 1]):
                number = int(''.join(line[number_start:j]))
                total += number
                ys, xs = np.where(is_star[i - 1: i + 2, number_start - 1: j + 1])
                for star_y, star_x in zip(ys.tolist(), xs.tolist()):
                    gears[(i - 1 + star_y, number_start - 1 + star_x)].append(number)
            number_start = None

print('part 1', total)
gear_ratio_sum = sum(
    gear[0] * gear[1]
    for gear in gears.values()
    if len(gear) == 2
)
print('part 2', gear_ratio_sum)

part 1 530849
part 2 84900879


# Day 4

In [218]:
import re

lines = read_input(day=4)
total = 0
counts = [1 for _ in range(len(lines))]


for index, line in enumerate(lines):
    winners, numbers = line.split(': ')[1].split(' | ')
    winners = set(re.split('\s+', winners))
    numbers = set(re.split('\s+', numbers))    
    winning_numbers = winners & numbers
    if winning_numbers:
        total += 1 << (len(winning_numbers) - 1)
    for offset in range(len(winning_numbers)):
        counts[index + offset + 1] += counts[index]
print('part 1', total)
print('part 2', sum(counts))

part 1 22674
part 2 5747443


# Day 5

In [331]:
lines = read_input(day=5)
traverse_order = ['seed']
maps = defaultdict(list)
initial_seeds = list(map(int, lines[0].split(': ')[1].split()))
index = 1
current_map = None
while index < len(lines):
    if not lines[index].strip():
        line = lines[index + 1]
        map_title = line.split()[0].split('-to-')
        current_map = tuple(map_title)
        traverse_order.append(map_title[1])
        index += 2
        continue
    maps[current_map].append(list(map(int, lines[index].split())))
    index += 1

def get_location(start_id):
    for mapping_type in zip(traverse_order, traverse_order[1:]):
        for mapping in maps[mapping_type]:
            if start_id >= mapping[1] and start_id < mapping[1] + mapping[2]:
                start_id = mapping[0] + (start_id - mapping[1])
                break
    return start_id
 

def get_location_range(current_ranges):
    for mapping_type in zip(traverse_order, traverse_order[1:]):
        result_ranges = []
        for dest_start_id, mapping_start_id, mapping_length in maps[mapping_type]:
            new_current_ranges = []
            mapping_end_id = mapping_start_id + mapping_length - 1
            for start_id, length in current_ranges:
                end_id = start_id + length - 1
                if end_id < mapping_start_id or mapping_end_id < start_id:
                    new_current_ranges.append((start_id, length))
                    continue
                overlap_start_id = max(mapping_start_id, start_id)
                overlap_end_id = min(mapping_end_id, end_id)
                if start_id < overlap_start_id:
                    new_current_ranges.append((start_id, overlap_start_id - start_id))
                if overlap_end_id < end_id:
                    new_current_ranges.append((overlap_end_id + 1, end_id - overlap_end_id - 1))
                    
                new_start_id = dest_start_id + (overlap_start_id - mapping_start_id)
                new_length = overlap_end_id - overlap_start_id + 1
                result_ranges.append((new_start_id, new_length))
            current_ranges = new_current_ranges
        current_ranges += result_ranges

    return min(start_id for start_id, length in current_ranges)

print('part 1', min(
    get_location(seed_id)
    for seed_id in initial_seeds
))
print('part 2', get_location_range([
    (initial_seeds[index], initial_seeds[index + 1])
    for index in range(0, len(initial_seeds), 2)
]))

part 1 218513636
part 2 81956384


# Day 6

In [348]:
lines = read_input(day=6)
times, goals = [list(map(int, x.split(':')[1].split())) for x in lines]

def ways_to_beat(race_time, race_goal):
    return sum(
        time_holding * (race_time - time_holding) > race_goal
        for time_holding in range(1, race_time)
    )
    
print('part 1', reduce(mul, (
    ways_to_beat(race_time, race_goal)
    for race_time, race_goal in zip(times, goals)
)))

race_time, race_goal = [int("".join(x.split(':')[1].split())) for x in lines]
print('part 2', ways_to_beat(race_time, race_goal))

part 1 6209190
part 2 28545089


# Day 7

In [399]:
from collections import Counter
lines = read_input(day=7)

JOKER_STRENGHT = 1

def hand_strength(hand, use_joker):
    face_cards = {card: index + 10 for index, card in enumerate(['T', 'J', 'Q', 'K', 'A'])}
    if use_joker:
        face_cards['J'] = JOKER_STRENGHT

    face_values = [face_cards[card] if card in face_cards else int(card) for card in hand]
    card_counts = Counter(face_values)
    sorted_card_counts = card_counts.most_common()
    if len(sorted_card_counts) != 1 and JOKER_STRENGHT in card_counts:
        joker_count = card_counts[JOKER_STRENGHT]
        sorted_card_counts = [count for count in sorted_card_counts if count[0] != JOKER_STRENGHT]
        sorted_card_counts[0] = (sorted_card_counts[0][0], sorted_card_counts[0][1] + joker_count)
    sortable_values = [count[1] for count in sorted_card_counts]
    return "".join([chr(ord('a') + x) for x in sortable_values + face_values])


def deck_sum(deck, use_joker):
    cards_and_bids = [
        (hand_strength(line.split()[0], use_joker), int(line.split()[1]))
        for line in deck
    ]
    return sum(
        (index + 1) * bid
        for index, (_, bid) in enumerate(sorted(cards_and_bids, key=lambda x: x[0]))
    )
    
print('part 1', deck_sum(lines, use_joker=False))
print('part 2', deck_sum(lines, use_joker=True))

part 1 251136060
part 2 249400220


# Day 8

In [541]:
import math

lines = read_input(day=8)
instructions, _, *mappings = lines
directions = {}

for mapping in mappings:
    source, destinations = mapping.split(' = ')
    dest_a, dest_b = destinations[1:-1].split(', ')
    directions[source] = (dest_a, dest_b)

def distance(start):
    index = 0
    current = start
    while not current.endswith('Z'):
        next_dir = 0 if instructions[index % len(instructions)] == 'L' else 1
        current = directions[current][next_dir]
        index += 1
    return index

print('part 1', distance('AAA'))
print('part 2', math.lcm(*[distance(node) for node in directions if node.endswith('A')]))

part 1 16043
part 2 15726453850399


# Day 9